In [1]:
# Import
import pandas as pd
import schedule
import time
from requests_oauthlib import OAuth1Session
import requests
import json
import snowflake.connector
from snowflake.connector.pandas_tools import write_pandas

In [ ]:
# Load in Protected Keys
with open('TripleSeat_API\API_Key.json') as fi:
    credentials = json.load(fi)
with open('TripleSeat_API\API_Secret.json') as fi2:
    credentials2 = json.load(fi2)

In [20]:
# Authenticate API
# Call Events API
# Drop Unnecessary Columns
# Loop Through Paginated "Results"
# Write to Dictionary
# Convert to DataFrame
def triple_test():
    TRIPLESEAT_OAUTH_CLIENT_KEY = credentials['TRIPLESEAT_OAUTH_CLIENT_KEY']
    TRIPLESEAT_OAUTH_CLIENT_SECRET = credentials2['TRIPLESEAT_OAUTH_CLIENT_SECRET']
    BASE_URL = 'https://api.tripleseat.com'
    client = OAuth1Session(TRIPLESEAT_OAUTH_CLIENT_KEY, client_secret=TRIPLESEAT_OAUTH_CLIENT_SECRET)
    r = client.get(BASE_URL + '/oauth/test_request')
    if r.status_code == 200:
        r = client.get(BASE_URL + '/v1/events.json')
    events_url='https://api.tripleseat.com/v1/events.json'
    events_api= client.get(events_url)
    events_json=events_api.json()
    key_list=['managers','contact','secondary_contacts','status_changes','attachments','lost_reason','lost_reason_details','selected_lead_sources','event_date_iso9601','event_start_utc','event_end_utc','event_start_iso8601','booking','owner','creator','updator','event_date_iso8601','event_end_iso8601','end_time','documents','account','offsite_address','document_ids','event_start','event_end','setup_time','teardown_time','event_start_time_with_setup_time','event_end_time_with_teardown_time','event_start_with_setup_iso8601','event_end_with_teardown_iso8601','event_style','guaranteed_guest_count','description','contact_id','account_id','owned_by','booking_id','updated_by','created_by','customer_id','site_id','room_ids','managing_user_ids','unassigned','event_type_id','post_as','document_ids','start_time','start_date','end_date','custom_fields','category_totals','billing_totals','lead']
    all_events=[]
    for page in range(1, events_json["total_pages"]):
        events_api=client.get(events_url + f"?page={page}")
        loop_json=events_api.json()['results']
        for i in range(len(loop_json)):
            for key in key_list:
                try:
                    del loop_json[i][key]
                except: 
                    continue
        all_events.extend(loop_json)
    df=pd.DataFrame(all_events)
    return df
df=triple_test()

df.to_csv('/root/TripleSeat_API/ThatGoodShit.csv')

In [ ]:
# Flatten Nested "Rooms" Column
bn=pd.DataFrame(df.rooms.values.tolist())
bn=pd.json_normalize(json.loads(bn.to_json(orient='records')))
# Define New DF
bn=bn[['0.id','0.name','0.capacity']]
# Concat Old DF and New DF
df_final=pd.concat([df,bn.reindex(df.index)],axis=1)
# Flatten Concated DF, Rename/Drop Columns
df_final=pd.json_normalize(json.loads(df_final.to_json(orient='records')))
df_final = df_final.rename(columns={'0.id':'room_id','0.name':'room_name','0.capacity':'room_capacity','location.id':'location_id','location.name':'location_name','location.site_id':'location_site_id'})
df_final = df_final.drop(['rooms','location.customer_id','location_id'],axis=1)
# Convert Column Names to Upper Case for Interface with Snowflake
df_final.columns = df_final.columns.str.upper()
# Split Timestamp Off Selected Columns for Data Type Conversion with Snowflake
df_final['CREATED_AT'] = df_final['CREATED_AT'].str.split(' ').str[0]
df_final['UPDATED_AT'] = df_final['UPDATED_AT'].str.split(' ').str[0]
df_final['DELETED_AT'] = df_final['DELETED_AT'].str.split(' ').str[0]
# Load in Protected SnowFlake Credentials
with open('TripleSeat_API\SnowFlakeUser.json') as fi3:
    credentials3 = json.load(fi3)
with open('TripleSeat_API\SnowFlakePass.json') as fi4:
    credentials4 = json.load(fi4)
with open('TripleSeat_API\SnowFlakeAccount.json') as fi5:
    credentials5 = json.load(fi5)
# Establish Connection with Snowflake
cnn = snowflake.connector.connect(
    user= credentials3['user'],
    password= credentials4['password']
    account = credentials5['account'],
    warehouse = 'COMPUTE_WH',
    database = 'capstone',
    schema = 'TripleSeat')
# Write Data to Snowflake
success, nchucks, nrows, _ = write_pandas(cnn,df_final,'TRIPLESEAT', quote_identifiers=True)

In [ ]:
def triple_test2():
    TRIPLESEAT_OAUTH_CLIENT_KEY = credentials['TRIPLESEAT_OAUTH_CLIENT_KEY']
    TRIPLESEAT_OAUTH_CLIENT_SECRET = credentials2['TRIPLESEAT_OAUTH_CLIENT_SECRET']
    BASE_URL = 'https://api.tripleseat.com'
    client = OAuth1Session(TRIPLESEAT_OAUTH_CLIENT_KEY, client_secret=TRIPLESEAT_OAUTH_CLIENT_SECRET)
    r = client.get(BASE_URL + '/oauth/test_request')
    if r.status_code == 200:
        r = client.get(BASE_URL + '/v1/events.json')
    leads_url='https://api.tripleseat.com/v1/leads.json'
    leads_api= client.get(leads_url)
    leads_json=leads_api.json()
    key_list1=['first_name','last_name','company','phone_number','phone_number_extension','email_address','additional_information','lead_form_id','contact_preference','campaign_name','campaign_source','booking_id','campaign_medium','campaign_term','campaign_content','owner','lead_source','deleted_at','customer_id','site_id','turned_down_at','turned_down_reason','booking_lead','contact_id','booking_id']
    all_leads=[]
    for page in range(1, leads_json["total_pages"]):
        leads_api=client.get(leads_url + f"?page={page}")
        loop_json=leads_api.json()['results']
        for i in range(len(loop_json)):
            for key in key_list1:
                try:
                    del loop_json[i][key]
                except: 
                    continue
        all_leads.extend(loop_json)
    df2=pd.DataFrame(all_leads)
    return df2
df_test2=triple_test2()

df_test2.to_csv('/root/TripleSeat_API/ThatGoodShit2.csv')